In [1]:
import pandas as pd
import re
from datetime import datetime

In [2]:
file_path = 'payment_08-Mar-2025.xlsx'

In [3]:
df = pd.read_excel(file_path)
#df.head()

In [4]:
# Select specific columns
df1 = df[['UID', 'BudgetItem', 'Department', 'Amount', 'Vendor/Transfer to department', 'AmountRemarks']]
#df1.head()

In [5]:
# Rename specific columns
df2 = df1.rename(columns={'UID': 'Entry No', 'BudgetItem': 'Dr Ledger Name', 'Department':'Dr Cost Center', 'Amount':'Dr Amt', 'Vendor/Transfer to department':'Cr Ledger Name' })
#df2.head()

In [6]:
# Modified function to extract Bill No., Invoice No., or Quotation Ref No.
def extract_reference(text):
    bill_match = re.search(r'Bill No\.\s*(\S+)', text)
    invoice_match = re.search(r'Invoice No\.\s*(\S+)', text)
    quotation_match = re.search(r'Quotation Ref No\.\s*(\S+)', text)
    
    if bill_match:
        return bill_match.group(1)
    elif invoice_match:
        return invoice_match.group(1)
    elif quotation_match:
        return quotation_match.group(1)
    
    return None

# Apply the function to your data frame
df2['Bill Ref No.'] = df2['AmountRemarks'].apply(extract_reference)

# Display the first 50 rows
#df2.head()

In [7]:
# Create the new 'Vch Narration' column by merging 'Entry No.' and 'AmountRemarks'
df2['Vch Narration'] = 'UID No. ' + df2['Entry No'].astype(str) + ' ' + df2['AmountRemarks']
# Display the updated dataframe
#df2.head()

In [8]:
df3=df2[['Entry No', 'Dr Ledger Name',	'Dr Cost Center', 'Dr Amt', 'Cr Ledger Name', 'Bill Ref No.', 'Vch Narration' ]]
#df3.head()

In [9]:
# Adding blank columns at specific positions
df3.insert(0, 'Date', '')  # Add 'Date' at the first position
df3.insert(2, 'Vch Name', '')  # Add 'Vch No' at the third position
df3.insert(7, 'Cr Cost Center', '')  # Add 'cost center' at the eighth position
df3.insert(8, 'Cr Amt', '')  # Add 'cost center' at the ninth position
# Display the updated dataframe
df3.head(5)

,Date,Entry No,Vch Name,Dr Ledger Name,Dr Cost Center,Dr Amt,Cr Ledger Name,Cr Cost Center,Cr Amt,Bill Ref No.,Vch Narration
0,,4240033638,,Y-EDU-Computer Lab,RDD-SBI-Gram-Seva-2022-23,500,Vipul Manohar Patil(United Solution),,,286,UID No. 4240033638 Computer Service Charges at...
1,,4240033634,,Y-LIVELIHOOD-Skill Development,RDD-SBI-Gram-Seva-2022-23,1700,Jitesh Baban Pagi,,,956,UID No. 4240033634 Traveling Expenses from Dol...
2,,4240033631,,Y-WaSH-Safe Drinking Water,RDD-SBI-Gram-Seva-2022-23,15812,Bhagirath Electricals & Borewell,,,106,UID No. 4240033631 Purchase of 0.5 Inch Water ...
3,,4240033630,,Y-WaSH-Safe Drinking Water,RDD-SBI-Gram-Seva-2022-23,16697,Bhagirath Electricals & Borewell,,,107,UID No. 4240033630 Purchase of 0.5 Inch Water ...
4,,4240033626,,Y-WaSH-Safe Drinking Water,RDD-SBI-Gram-Seva-2022-23,9794,Bhagirath Electricals & Borewell,,,111,UID No. 4240033626 Purchase of 0.5 Inch Water ...


In [10]:
df4 = df3[['Date','Entry No','Vch Name','Dr Ledger Name','Dr Amt','Dr Cost Center','Cr Ledger Name','Cr Amt','Cr Cost Center','Vch Narration','Bill Ref No.']]
df4.head(5)

,Date,Entry No,Vch Name,Dr Ledger Name,Dr Amt,Dr Cost Center,Cr Ledger Name,Cr Amt,Cr Cost Center,Vch Narration,Bill Ref No.
0,,4240033638,,Y-EDU-Computer Lab,500,RDD-SBI-Gram-Seva-2022-23,Vipul Manohar Patil(United Solution),,,UID No. 4240033638 Computer Service Charges at...,286
1,,4240033634,,Y-LIVELIHOOD-Skill Development,1700,RDD-SBI-Gram-Seva-2022-23,Jitesh Baban Pagi,,,UID No. 4240033634 Traveling Expenses from Dol...,956
2,,4240033631,,Y-WaSH-Safe Drinking Water,15812,RDD-SBI-Gram-Seva-2022-23,Bhagirath Electricals & Borewell,,,UID No. 4240033631 Purchase of 0.5 Inch Water ...,106
3,,4240033630,,Y-WaSH-Safe Drinking Water,16697,RDD-SBI-Gram-Seva-2022-23,Bhagirath Electricals & Borewell,,,UID No. 4240033630 Purchase of 0.5 Inch Water ...,107
4,,4240033626,,Y-WaSH-Safe Drinking Water,9794,RDD-SBI-Gram-Seva-2022-23,Bhagirath Electricals & Borewell,,,UID No. 4240033626 Purchase of 0.5 Inch Water ...,111


In [11]:
# Ensure 'Dr Amt' is a string and remove commas
df3['Dr Amt'] = df3['Dr Amt'].astype(str).str.replace(',', '', regex=True)
# Convert 'Dr Amt' to numeric
df3['Dr Amt'] = pd.to_numeric(df3['Dr Amt'])
# Create a copy of df3 to df4
df4 = df3.copy()
# Calculate cumulative sum for each 'Entry No'
df4['Cr Amt'] = df3.groupby('Entry No')['Dr Amt'].transform('sum')
# Format 'Dr Amt' and 'Cr Amt' with commas and two decimal places
df4['Dr Amt'] = df4['Dr Amt'].apply(lambda x: f"{x:,.2f}")
df4['Cr Amt'] = df4['Cr Amt'].apply(lambda x: f"{x:,.2f}")
# Display the final DataFrame
df4.head(5)

C:\Users\grd00\AppData\Local\Temp\ipykernel_24120\2643644666.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3['Dr Amt'] = df3['Dr Amt'].astype(str).str.replace(',', '', regex=True)
C:\Users\grd00\AppData\Local\Temp\ipykernel_24120\2643644666.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3['Dr Amt'] = pd.to_numeric(df3['Dr Amt'])


,Date,Entry No,Vch Name,Dr Ledger Name,Dr Cost Center,Dr Amt,Cr Ledger Name,Cr Cost Center,Cr Amt,Bill Ref No.,Vch Narration
0,,4240033638,,Y-EDU-Computer Lab,RDD-SBI-Gram-Seva-2022-23,500.00,Vipul Manohar Patil(United Solution),,500.00,286,UID No. 4240033638 Computer Service Charges at...
1,,4240033634,,Y-LIVELIHOOD-Skill Development,RDD-SBI-Gram-Seva-2022-23,"1,700.00",Jitesh Baban Pagi,,"1,700.00",956,UID No. 4240033634 Traveling Expenses from Dol...
2,,4240033631,,Y-WaSH-Safe Drinking Water,RDD-SBI-Gram-Seva-2022-23,"15,812.00",Bhagirath Electricals & Borewell,,"15,812.00",106,UID No. 4240033631 Purchase of 0.5 Inch Water ...
3,,4240033630,,Y-WaSH-Safe Drinking Water,RDD-SBI-Gram-Seva-2022-23,"16,697.00",Bhagirath Electricals & Borewell,,"16,697.00",107,UID No. 4240033630 Purchase of 0.5 Inch Water ...
4,,4240033626,,Y-WaSH-Safe Drinking Water,RDD-SBI-Gram-Seva-2022-23,"9,794.00",Bhagirath Electricals & Borewell,,"9,794.00",111,UID No. 4240033626 Purchase of 0.5 Inch Water ...


In [12]:
tally_icm_heads = pd.read_excel('tally_icm_heads.xlsx')
tally_icm_heads = tally_icm_heads[['icm_heads','tally_heads']]
tally_icm_heads = tally_icm_heads.rename(columns={'icm_heads':'Dr Ledger Name'})
tally_icm_heads.head(5)

,Dr Ledger Name,tally_heads
0,Y-ADMN-Administration Expenses,ADMN-Administrative Expenses
1,Y1-ADMN-Administrative Expenses,ADMN-Administrative Expenses
2,ADMN-Administration Expenses,ADMN-Administrative Expenses
3,Y-ADMN-Administrative Expenses,ADMN-Administrative Expenses
4,Y-ADMN-Administrative Expenses,ADMN-Administrative Expenses


In [13]:
# Use merge instead of map to handle duplicate keys
df4 = df4.merge(tally_icm_heads, on='Dr Ledger Name', how='left')
# Drop the old 'Dr Ledger Name' column if necessary
df4.drop(columns=['Dr Ledger Name'], inplace=True)
# Rename 'tally_heads' to 'Dr Ledger Name'
df4.rename(columns={'tally_heads': 'Dr Ledger Name'}, inplace=True)
df4.head(5)

,Date,Entry No,Vch Name,Dr Cost Center,Dr Amt,Cr Ledger Name,Cr Cost Center,Cr Amt,Bill Ref No.,Vch Narration,Dr Ledger Name
0,,4240033638,,RDD-SBI-Gram-Seva-2022-23,500.00,Vipul Manohar Patil(United Solution),,500.00,286,UID No. 4240033638 Computer Service Charges at...,NaN
1,,4240033634,,RDD-SBI-Gram-Seva-2022-23,"1,700.00",Jitesh Baban Pagi,,"1,700.00",956,UID No. 4240033634 Traveling Expenses from Dol...,LIVELIHOOD-Skill Development
2,,4240033634,,RDD-SBI-Gram-Seva-2022-23,"1,700.00",Jitesh Baban Pagi,,"1,700.00",956,UID No. 4240033634 Traveling Expenses from Dol...,LIVELIHOOD-Skill Development
3,,4240033634,,RDD-SBI-Gram-Seva-2022-23,"1,700.00",Jitesh Baban Pagi,,"1,700.00",956,UID No. 4240033634 Traveling Expenses from Dol...,LIVELIHOOD-Skill Development
4,,4240033634,,RDD-SBI-Gram-Seva-2022-23,"1,700.00",Jitesh Baban Pagi,,"1,700.00",956,UID No. 4240033634 Traveling Expenses from Dol...,LIVELIHOOD-Skill Development


In [14]:
df5 = df4[['Date','Entry No','Vch Name','Dr Ledger Name','Dr Cost Center','Dr Amt','Cr Ledger Name','Cr Cost Center','Cr Amt','Bill Ref No.','Vch Narration']]
df5.tail()

,Date,Entry No,Vch Name,Dr Ledger Name,Dr Cost Center,Dr Amt,Cr Ledger Name,Cr Cost Center,Cr Amt,Bill Ref No.,Vch Narration
114,,4240033400,,PRM-Food Expenses,RDD-REP-SCHBANG,300.00,Mahalaxmi Cloth Store Pro Jain,,300.00,None,UID No. 4240033400 Purchases of Carry Bags for...
115,,4240033400,,PRM-Food Expenses,RDD-REP-SCHBANG,300.00,Mahalaxmi Cloth Store Pro Jain,,300.00,None,UID No. 4240033400 Purchases of Carry Bags for...
116,,4240033400,,PRM-Food Expenses,RDD-REP-SCHBANG,300.00,Mahalaxmi Cloth Store Pro Jain,,300.00,None,UID No. 4240033400 Purchases of Carry Bags for...
117,,4240033400,,PRM-Food Expenses,RDD-REP-SCHBANG,300.00,Mahalaxmi Cloth Store Pro Jain,,300.00,None,UID No. 4240033400 Purchases of Carry Bags for...
118,,4240033400,,PRM-Food Expenses,RDD-REP-SCHBANG,300.00,Mahalaxmi Cloth Store Pro Jain,,300.00,None,UID No. 4240033400 Purchases of Carry Bags for...


In [15]:
df6 = df5.drop_duplicates()
df6.tail()

,Date,Entry No,Vch Name,Dr Ledger Name,Dr Cost Center,Dr Amt,Cr Ledger Name,Cr Cost Center,Cr Amt,Bill Ref No.,Vch Narration
86,,4240033405,,ADMN-Administrative Expenses,RDD-REP-SCHBANG,"1,040.00",MSEDCL-003464,,"1,040.00",000002779730586,UID No. 4240033405 Electricity Bill Charges fo...
93,,4240033404,,PRM-Food Expenses,RDD-REP-SCHBANG,"51,784.00",Daadan Foods LLP,,"51,784.00",1011,UID No. 4240033404 Purchase of Grocery for Ref...
99,,4240033403,,ADMN-Administrative Expenses,RDD-REP-STERIMAX,260.00,MSEDCL-003464,,260.00,None,UID No. 4240033403 Electricity Bill Charges fo...
106,,4240033402,,ADMN-Administrative Expenses,RDD-REP-STERIMAX,400.00,MSEDCL-003464,,400.00,000002779730353,UID No. 4240033402 Electricity Bill Charges fo...
113,,4240033400,,PRM-Food Expenses,RDD-REP-SCHBANG,300.00,Mahalaxmi Cloth Store Pro Jain,,300.00,None,UID No. 4240033400 Purchases of Carry Bags for...


In [16]:
# Get current date and timestamp
current_time = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
# Create output file name with date and timestamp
output_filename = f'output_{current_time}.xlsx'
# Convert DataFrame to Excel
df6.to_excel(output_filename, index=False)